In [1]:
# import sys; sys.path.append('common/');
from helper import *

DATA_PATH = '/projects/persuasionforgood-master/Face_acts/dialogue_act_prediction/resisting-persuasion/data'

df = pd.read_csv(DATA_PATH+'/Final_annotations.csv')
df = df.rename(columns={"Our Label": "fine_labels"})

for index, row in df.iterrows():
    if row['B4']==1 and row['fine_labels']!= row['fine_labels']:
        print(index, row)


fine_labels = [i for i in list(set(df['fine_labels'])) if i==i]

fine2coarse_dict = {}
fine2coarse_dict['deflect-responsibility'] = 'contesting'
fine2coarse_dict['organization-inquiry']   = 'contesting'#'biased-processing'
fine2coarse_dict['attack-credibility']     = 'contesting'
fine2coarse_dict['self-pity']              = 'empowerment'
fine2coarse_dict['nitpicking']             = 'contesting'
fine2coarse_dict['direct-rejection']       = 'avoidance'
fine2coarse_dict['personal-choice']        = 'empowerment'
fine2coarse_dict['delay-tactic']           = 'avoidance'
fine2coarse_dict['hesitance']              = 'avoidance'
fine2coarse_dict['not-a-strategy']         = 'not-a-strategy'

fine2resistance_dict = {}
fine2resistance_dict['deflect-responsibility'] = 'counter-argumentation'
fine2resistance_dict['organization-inquiry']   = 'information-inquiry' # biased-processing
fine2resistance_dict['attack-credibility']     = 'source-degradation'
fine2resistance_dict['self-pity']              = 'self-pity'
fine2resistance_dict['nitpicking']             = 'counter-argumentation'
fine2resistance_dict['direct-rejection']       = 'self-assertion'
fine2resistance_dict['personal-choice']        = 'personal-choice'
fine2resistance_dict['delay-tactic']           = 'selective-avoidance'
fine2resistance_dict['hesitance']              = 'selective-avoidance'
fine2resistance_dict['not-a-strategy']         = 'not-a-strategy'

coarse_labels     = []
resistance_labels = []

df['fine_labels'] = df['fine_labels'].fillna('not-a-strategy')
for i, row in df.iterrows():
    label = row['fine_labels']
    if label in fine2coarse_dict:
        coarse_labels.append(fine2coarse_dict[label])
        resistance_labels.append(fine2resistance_dict[label])
    else:
        resistance_labels.append('not-a-strategy')
        coarse_labels.append('not-a-strategy')
        
df['coarse_labels'] = coarse_labels
df['resistance_labels'] = resistance_labels

df.to_csv(DATA_PATH+'/data.csv')
info_df = pd.read_csv(DATA_PATH+'/300_info.csv',sep=',')

sincere_donors_ids=set()
sincere_nondonors_ids=set()
for index, row in info_df.iterrows():
    did= row['B2']
    role=str(row['B4'])
    prop_amt=float(row['B5'])
    amt= float(row['B6'])
    if role =='1':
        if amt>0 and prop_amt<= amt:
            sincere_donors_ids.add(did)
        elif amt==0 and (prop_amt== 0 or math.isnan(prop_amt)):
            sincere_nondonors_ids.add(did)
        elif amt>0 and prop_amt> amt:
            sincere_donors_ids.add(did)
        else:
            # print(amt,prop_amt)
#             sincere_donors_ids.add(did)
            sincere_nondonors_ids.add(did)

print("Sincere donors= ",len(sincere_donors_ids))
print("Sincere non-donors= ",len(sincere_nondonors_ids))

Sincere donors=  115
Sincere non-donors=  185


In [2]:
def cohen_d(x,y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)

In [3]:
print('No of valid data-points {}'.format(len(df[df.B4==1])))

import scipy.stats

def group_name_dict(id_):
    if id_ in sincere_donors_ids:
        return 'SD'
    elif id_ in sincere_nondonors_ids:
        return 'SND'

def print_label_dist(label_name):
    labels_list  = list(set(list(df[df.B4==1][label_name])))
    labels_dict  = ddict(lambda: ddict(int))
    convs_dict   = ddict(lambda: ddict(list))
        
    df2 = df[df.B4 ==1]
    curr_id = list(df2['B2'])[0]
    temp_labels_dict = ddict(int)
    
    for index, row in df2.iterrows():
        if curr_id != row['B2']:
            group_name = group_name_dict(curr_id)
            sum_labels_list = np.sum([temp_labels_dict[i] for i in temp_labels_dict])
            for label in labels_list:
                if label in temp_labels_dict:
                    convs_dict[group_name][label].append(temp_labels_dict[label]/sum_labels_list)
                else:
                    convs_dict[group_name][label].append(0)
            curr_id = row['B2']
            temp_labels_dict = ddict(int)
        temp_labels_dict[row[label_name]]+=1
        labels_dict[group_name_dict(row['B2'])][row[label_name]]+=1
    
    # meant for the last index which would have been missed otherwise
    group_name = group_name_dict(curr_id)
    sum_labels_list = np.sum([temp_labels_dict[i] for i in temp_labels_dict])
    for label in labels_list:
        if label in temp_labels_dict:
            convs_dict[group_name][label].append(temp_labels_dict[label]/sum_labels_list)
        else:
            convs_dict[group_name][label].append(0)

    print('Distribution of the different strategies among the different categories')
    print("Label,#Donor,#NonDonor,Mean-D,Mean-ND,pval-1,pval-2,Cohen-d")

#     print("Label\t\t\t#Donor #NonDonor Mean-D Mean-ND pval-1\t pval-2\t Cohen-d")
    for label in labels_list:
        a = convs_dict['SD'][label]
        b = convs_dict['SND'][label]
        mu_a = round(np.mean(a),3)
        mu_b = round(np.mean(b),3)
        sum_a = labels_dict['SD'][label]
        sum_b = labels_dict['SND'][label]
        stat, pval   = scipy.stats.ks_2samp(a,b)
        stat2,pval2  = scipy.stats.ttest_ind(a,b,equal_var=False)
        effect_size  = round(cohen_d(a,b),4)
#         if len(label)<=len('personal-choice'):
#             print("{}\,\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(label, sum_a, sum_b, mu_a , mu_b, round(pval,4), round(pval2,4), effect_size))
#         else:
#             print("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(label, sum_a, sum_b, mu_a , mu_b, round(pval,4), round(pval2,4), effect_size))
        print("{},{},{},{},{},{},{},{}".format(label, sum_a, sum_b, mu_a , mu_b, round(pval,4), round(pval2,4), effect_size))
    print('*********************************\n\n')
    return 

No of valid data-points 4638


In [4]:
print_label_dist('fine_labels')

print_label_dist('coarse_labels')

print_label_dist('resistance_labels')

Distribution of the different strategies among the different categories
Label,#Donor,#NonDonor,Mean-D,Mean-ND,pval-1,pval-2,Cohen-d
direct-rejection,6,45,0.004,0.015,0.2749,0.007,-0.2925
not-a-strategy,1482,2017,0.799,0.717,0.0011,0.0012,0.3851
deflect-responsibility,38,73,0.021,0.028,0.9732,0.2926,-0.1199
organization-inquiry,137,322,0.088,0.117,0.0072,0.0763,-0.2145
personal-choice,42,104,0.023,0.036,0.907,0.0662,-0.2026
attack-credibility,44,100,0.025,0.033,0.5991,0.3106,-0.1186
hesitance,9,26,0.004,0.01,0.991,0.0557,-0.2001
nitpicking,13,30,0.008,0.009,1.0,0.9187,-0.0121
delay-tactic,16,40,0.011,0.014,0.3727,0.5072,-0.0844
self-pity,31,63,0.017,0.022,0.8089,0.4917,-0.0871
*********************************


Distribution of the different strategies among the different categories
Label,#Donor,#NonDonor,Mean-D,Mean-ND,pval-1,pval-2,Cohen-d
contesting,232,525,0.142,0.186,0.0461,0.0343,-0.2518
not-a-strategy,1482,2017,0.799,0.717,0.0011,0.0012,0.3851
empowerment,73,167,0.04,0.057,0.3727

In [5]:
fine_labels_dict = ddict(int)
coarse_labels_dict = ddict(int)
resistance_labels_dict = ddict(int)

fine_labels_list = [i for i in list(set(df['fine_labels'])) if i==i]
coarse_labels_list = [i for i in list(set(df['coarse_labels'])) if i!='None']
resistance_labels_list = [i for i in list(set(df['resistance_labels'])) if i!='None' ]

for i, elem in enumerate(fine_labels_list):
    fine_labels_dict[elem]=i
for i, elem in enumerate(coarse_labels_list):
    coarse_labels_dict[elem]=i
for i, elem in enumerate(resistance_labels_list):
    resistance_labels_dict[elem]=i

# {'nitpicking': 0, 'direct-rejection': 1, 'personal-choice': 2, 'self-pity': 3, 'attack-credibility': 4, 'organization-inquiry': 5, 'hesitance': 6, 'deflect-responsibility': 7, 'delay-tactic': 8, 'not-a-strategy': 9}

print(fine_labels_dict)
print(coarse_labels_dict)
print(resistance_labels_dict)

defaultdict(<class 'int'>, {'direct-rejection': 0, 'not-a-strategy': 1, 'deflect-responsibility': 2, 'organization-inquiry': 3, 'personal-choice': 4, 'attack-credibility': 5, 'hesitance': 6, 'nitpicking': 7, 'delay-tactic': 8, 'self-pity': 9})
defaultdict(<class 'int'>, {'contesting': 0, 'not-a-strategy': 1, 'empowerment': 2, 'avoidance': 3})
defaultdict(<class 'int'>, {'not-a-strategy': 0, 'source-degradation': 1, 'personal-choice': 2, 'self-assertion': 3, 'information-inquiry': 4, 'selective-avoidance': 5, 'counter-argumentation': 6, 'self-pity': 7})


In [6]:
Ds= list(set(sincere_donors_ids))
NDs = list(set(sincere_nondonors_ids))
alls = list(set(df['B2']))
test_ids_dict = ddict(list)
train_ids_dict = ddict(list)

for count in range(0,5):
    test_ids = [id_ for elem, id_ in enumerate(Ds) if elem%5 ==count]
    test_ids.extend([id_ for elem, id_ in enumerate(NDs) if elem%5 ==count])
    train_ids = [id_ for id_ in alls if id_ not in test_ids]
    random.shuffle(test_ids)
    random.shuffle(train_ids)
    train_ids_dict[count] = train_ids
    test_ids_dict[count]  = test_ids
    

info_dict = {}
info_dict['fine_labels'] = fine_labels_dict
info_dict['coarse_labels'] = coarse_labels_dict
info_dict['resistance_labels'] = resistance_labels_dict
info_dict['train_ids'] = train_ids_dict
info_dict['test_ids']  = test_ids_dict

with open(DATA_PATH+'info_dict.pt','rb') as handle:
    info_dict= pickle.load(handle)

In [22]:
def process_cnn_text(text):
    return text.replace('\t',' ').lower().strip()


for count in range(0,5):
    print('count {}'.format(count), end ='\r')
    train_ids = train_ids_dict[count]
    test_ids  = test_ids_dict[count]
    
    train= df.loc[(df['B2'].isin(train_ids)) & (df['B4']==1)]
    test= df.loc[(df['B2'].isin(test_ids)) & (df['B4']==1)]
    
    coarse_train_csv = open(DATA_PATH+'/train/coarse_train'+str(count)+'.txt','w')
    coarse_test_csv = open(DATA_PATH+'/test/coarse_test'+str(count)+'.txt','w')
    
    for index, row in train.iterrows():
        coarse_train_csv.write(process_cnn_text(row['Unit'])+'\t'+ str(info_dict['coarse_labels'][row['coarse_labels']])+'\n')
        
    for index, row in test.iterrows():
        coarse_test_csv.write(process_cnn_text(row['Unit'])+'\t'+ str(info_dict['coarse_labels'][row['coarse_labels']])+'\n')
        
    coarse_train_csv.close()
    coarse_test_csv.close()
        
    fine_train_csv = open(DATA_PATH+'/train/fine_train'+str(count)+'.txt','w')
    fine_test_csv = open(DATA_PATH+'/test/fine_test'+str(count)+'.txt','w')
    
    for index, row in train.iterrows():
        fine_train_csv.write(process_cnn_text(row['Unit'])+'\t'+ str(info_dict['fine_labels'][row['fine_labels']])+'\n')
        
    for index, row in test.iterrows():
        fine_test_csv.write(process_cnn_text(row['Unit'])+'\t'+ str(info_dict['fine_labels'][row['fine_labels']])+'\n')

    fine_train_csv.close()
    fine_test_csv.close()
        
    resistance_train_csv = open(DATA_PATH+'/train/resistance_train'+str(count)+'.txt','w')
    resistance_test_csv = open(DATA_PATH+'/test/resistance_test'+str(count)+'.txt','w')
    
    for index, row in train.iterrows():
        resistance_train_csv.write(process_cnn_text(row['Unit'])+'\t'+ str(info_dict['resistance_labels'][row['resistance_labels']])+'\n')
        
    for index, row in test.iterrows():
        resistance_test_csv.write(process_cnn_text(row['Unit'])+'\t'+ str(info_dict['resistance_labels'][row['resistance_labels']])+'\n')

    resistance_train_csv.close()
    resistance_test_csv.close()

In [23]:
display(df.tail())

,B2,B4,Turn,Unit,er_label_1,ee_label_1,fine_labels,coarse_labels,resistance_labels
10387,20180827-050740_909_live,1,8,That really wouldn't bother me at all.,NaN,negative-reaction-to-donation,personal-choice,empowerment,personal-choice
10388,20180827-050740_909_live,0,9,"well like I said, any donation is a good donat...",logical-appeal,NaN,not-a-strategy,not-a-strategy,not-a-strategy
10389,20180827-050740_909_live,0,9,thank you for donating,thank,NaN,not-a-strategy,not-a-strategy,not-a-strategy
10390,20180827-050740_909_live,1,9,I'm not donating to your children's charity.,NaN,disagree-donation,direct-rejection,avoidance,self-assertion
10391,20180827-050740_909_live,1,9,How much are you donating?,NaN,ask-persuader-donation-intention,deflect-responsibility,contesting,counter-argumentation


In [7]:
import sys, importlib
importlib.reload(sys.modules['helper'])
from helper import *

## create data jsons for the Hi-GRU model

In [8]:
affect_file= '../data/Resources/NRC_affect.txt'
affect_df= pd.read_csv(affect_file, sep='\t', lineterminator='\n')
display(affect_df.head())

print(set(affect_df['AffectDimension']))

vad_file= '../data/Resources/NRC_vad.txt'
vad_df= pd.read_csv(vad_file, sep='\t', lineterminator='\n')

display(vad_df.head())

emotion_file= '../data/Resources/NRC_emotion.txt'
emo_df= pd.read_csv(emotion_file, sep='\t', lineterminator='\n')
display(emo_df.head())
print(set(emo_df['Emotion']))

with open('../data/Resources/word_categories.p','rb') as handle:
    word_categories_dict= pickle.load(handle)

with open('../data/Resources/starred_word_categories.p','rb')as handle:
    starred_word_categories_dict= pickle.load(handle)

liwc_categories=['Funct','Pronoun','Ppron','I','We','You','SheHe','They','Ipron','Article','Verbs','AuxVb','Past','Present','Future','Adverbs','Prep','Conj','Negate','Quant','Numbers','Swear','','Family','Friends','Humans','Affect','Posemo','Negemo','Anx','Anger','Sad','CogMech','Insight','Cause','Discrep','Tentat','Certain','Inhib','Incl','Excl','Percept','See','Hear','Feel','Body','Health','Sexual','Ingest','Relativ','Motion','Space','Time','Work','Achiev','Leisure','Home','Money','Relig','Death','Assent','Nonflu','Filler','Other']	

print(len(liwc_categories))

def create_vad_dict(vad_df):
    vad_dict={}
    for index, row in vad_df.iterrows():
        w= row['Word']
        vad_dict[w]={}
        vad_dict[w]['Valence']=row['Valence']
        vad_dict[w]['Arousal']= row['Arousal']
        vad_dict[w]['Dominance']= row['Dominance']
    
    return vad_dict

def create_emo_dict(emo_df):
    emo_dict={}
    emotions_set=list({'trust', 'fear', 'surprise', 'sadness', 'positive', 'joy', 'anger', 'negative', 'disgust', 'anticipation'})
    for index, row in emo_df.iterrows():
        w= row['Word']
        if w not in emo_dict:
            emo_dict[w]={}
            for emotion in emotions_set:
                emo_dict[w][emotion]=0
        emo_dict[w][row['Emotion']]= row['Presence']
    
    return emo_dict

def create_affect_dict(affect_df):
    affect_dict={}
    affect_set= list({'joy', 'sadness', 'fear', 'anger'})
    for index, row in affect_df.iterrows():
        w= row['term']
        if w not in affect_dict:
            affect_dict[w]={}
            for affect in affect_set:
                affect_dict[w][affect]=0
        affect_dict[w][row['AffectDimension']]= row['score']
    return affect_dict

vad_dict= create_vad_dict(vad_df)
emo_dict= create_emo_dict(emo_df)
affect_dict= create_affect_dict(affect_df)


import numpy as np

def create_embeddings(word_list, vads, affects, emos):
    
    vad_sent_dict={}
    affect_sent_dict={}
    emo_sent_dict={}
    
    for elem in vads:
        vad_sent_dict[elem]=[]
    for elem in affects:
        affect_sent_dict[elem]=[]
    for elem in emos:
        emo_sent_dict[elem]=[]
    
    for w in word_list:
        if w in vad_dict:
            for elem in vads:
                try:
                    vad_sent_dict[elem].append(float(vad_dict[w][elem]))
                except Exception as e:
                    vad_sent_dict[elem].append(0)
        else:
            for elem in vads:
                vad_sent_dict[elem].append(0)
        
        if w in affect_dict:
            for elem in affects:
                try:
                    affect_sent_dict[elem].append(float(affect_dict[w][elem]))
                except Exception as e:
                    affect_sent_dict[elem].append(0)
        else:
            for elem in affects:
                affect_sent_dict[elem].append(0)
        
        if w in emo_dict:
            for elem in emos:
                try:
                    emo_sent_dict[elem].append(float(emo_dict[w][elem]))
                except Exception as e:
                    emo_sent_dict[elem].append(0)
        else:
            for elem in emos:
                emo_sent_dict[elem].append(0)
            
    vad_features    = []
    affect_features = []
    emo_features    = []
    
    for elem in vads:
        val = np.mean(vad_sent_dict[elem])
        if val== val:
            vad_features.append(val)
        else:
            vad_features.append(0)
            
    for elem in affects:
        val = np.mean(affect_sent_dict[elem])
        if val== val:
            affect_features.append(val)
        else:
            affect_features.append(0)
    for elem in emos:
        val = np.mean(emo_sent_dict[elem])
        if val== val:
            emo_features.append(val)
        else:
            emo_features.append(0)
        
    return vad_features, affect_features, emo_features

def create_liwc(words):
    category_tag={}
    count=0
    for category in liwc_categories:
        category_tag[category]=0
    category_tag['Other']=0	

    for word in words:
        flag1=0
        flag2=0

        if word_categories_dict[word]!=[]:
            flag1=1

        for category in word_categories_dict[word]:
            if category not in category_tag:
                category_tag[category]=1
            else:
                category_tag[category]+=1

        for substring in starred_word_categories_dict.keys():
            if word.startswith(substring):
                flag2=1
                for category in starred_word_categories_dict[substring]:
                    if category not in category_tag:
                        category_tag[category]=1
                    else:
                        category_tag[category]+=1

        if flag1==0 and flag2==0:
            category_tag['Other']+=1
        count+=1

    if count>0:
        for i in category_tag:
            category_tag[i]=category_tag[i]/count

    return category_tag

vads=['Valence', 'Arousal', 'Dominance']
affects=sorted(list({'joy', 'sadness', 'fear', 'anger'}))
emos=sorted(list({'trust', 'fear', 'surprise', 'sadness', 'positive', 'joy', 'anger', 'negative', 'disgust', 'anticipation'}))
liwc_categories=['Funct','Pronoun','Ppron','I','We','You','SheHe','They','Ipron','Article','Verbs','AuxVb','Past','Present','Future','Adverbs','Prep','Conj','Negate','Quant','Numbers','Swear','','Family','Friends','Humans','Affect','Posemo','Negemo','Anx','Anger','Sad','CogMech','Insight','Cause','Discrep','Tentat','Certain','Inhib','Incl','Excl','Percept','See','Hear','Feel','Body','Health','Sexual','Ingest','Relativ','Motion','Space','Time','Work','Achiev','Leisure','Home','Money','Relig','Death','Assent','Nonflu','Filler','Other']	

,term,score,AffectDimension
0,outraged,0.964,anger
1,brutality,0.959,anger
2,hatred,0.953,anger
3,hateful,0.940,anger
4,terrorize,0.939,anger


{'sadness', 'fear', 'anger', 'joy'}


,Word,Valence,Arousal,Dominance
0,aaaaaaah,0.479,0.606,0.291
1,aaaah,0.520,0.636,0.282
2,aardvark,0.427,0.490,0.437
3,aback,0.385,0.407,0.288
4,abacus,0.510,0.276,0.485


,Word,Emotion,Presence
0,aback,anger,0
1,aback,anticipation,0
2,aback,disgust,0
3,aback,fear,0
4,aback,joy,0


{'positive', 'disgust', 'negative', 'trust', 'fear', 'anger', 'sadness', 'surprise', 'anticipation', 'joy'}
64


In [9]:
face_act_dir = '/projects/persuasionforgood-master/Face_acts/data/'

df1 = pd.read_csv(face_act_dir+'train0.csv')
df2 = pd.read_csv(face_act_dir+'test0.csv')
df3 = pd.concat([df1,df2])

face_acts  = ['spos+','spos-','sneg+','hpos+','hpos-','hneg+','hneg-','other']
sentiments = ['pos','neg','neu']
er_strategies  = ["logical-appeal","credibility-appeal","self-modeling","foot-in-the-door","source-related-inquiry","emotion-appeal","personal-related-inquiry","donation-information","task-related-inquiry","personal-story"]
er_DAs     = [i for i in list(set(list(df['er_label_1']))) if i==i and i not in er_strategies]
ee_DAs     = list(set([i for i in list(set(list(df['ee_label_1']))) if i ==i]))

print(ee_DAs)
print(er_DAs)
print(er_strategies)

df3.to_csv('../data/face_act.csv')


['positive-reaction-to-donation', 'task-related-inquiry', 'acknowledgement', 'greeting', 'agree-donation', 'disagree-donation', 'negative-to-inquiry', 'closing', 'you-are-welcome', 'provide-donation-amount', 'neutral-reaction-to-donation', 'personal-related-inquiry', 'disagree-donation-more', 'thank', 'positive-to-inquiry', 'negative-reaction-to-donation', 'ask-donation-procedure', 'ask-org-info', 'neutral-to-inquiry', 'off-task', 'confirm-donation', 'ask-persuader-donation-intention', 'other']
['ask-not-donate-reason', 'greeting', 'acknowledgement', 'negative-to-inquiry', 'closing', 'you-are-welcome', 'thank', 'positive-to-inquiry', 'ask-donation-amount', 'proposition-of-donation', 'neutral-to-inquiry', 'praise-user', 'off-task', 'confirm-donation', 'ask-donate-more', 'other', 'comment-partner']
['logical-appeal', 'credibility-appeal', 'self-modeling', 'foot-in-the-door', 'source-related-inquiry', 'emotion-appeal', 'personal-related-inquiry', 'donation-information', 'task-related-inqu

In [16]:
print(sincere_donors_ids)

{'20180723-025150_733_live', '20180826-174120_426_live', '20180825-210735_192_live', '20180808-013755_350_live', '20180825-060251_387_live', '20180808-011921_673_live', '20180825-075630_493_live', '20180808-052501_689_live', '20180826-062755_702_live', '20180827-033451_673_live', '20180825-070900_49_live', '20180826-054028_444_live', '20180719-211253_768_live', '20180826-174957_757_live', '20180826-180631_700_live', '20180825-081402_621_live', '20180825-044048_639_live', '20180826-082717_200_live', '20180903-103939_512_live', '20180827-032731_796_live', '20180826-181951_904_live', '20180723-061928_535_live', '20180825-080802_964_live', '20180723-052203_936_live', '20180808-034352_717_live', '20180903-163331_887_live', '20180904-024226_703_live', '20180723-114415_254_live', '20180825-071727_819_live', '20180904-025026_901_live', '20180826-184037_209_live', '20180826-212714_597_live', '20180825-063117_677_live', '20180826-182522_887_live', '20180825-053711_81_live', '20180717-200206_41_l

In [17]:
import re
only_words = r'\b\w+\b'

exception_arr = []

def normalize_arr(arr):
    if np.sum(arr)>0:
        return arr/np.sum(arr)
    else:
        return np.zeros(len(arr))

def create_conv(temp_df, train_flag = True):
    try:
        curr_id      = list(temp_df['B2'])[0]
    except Exception as e:
        import pdb; pdb.set_trace()
    

    convs      = []
    conv       = []
    max_utt_len  = 0
    er_strategies_feats = np.zeros(len(er_strategies))
    er_DAs_feats        = np.zeros(len(er_DAs))
    ee_DAs_feats        = np.zeros(len(ee_DAs))
    
    print(temp_df.columns)

    max_utt_len = 0 
    for index, row in temp_df.iterrows():
        print('Done for {}'.format(index), end='\r')
        if row['B2']!= curr_id:
            for elem in conv:
                elem['bert-feat'].extend([0 for i in range(max_utt_len-len(elem['bert-feat']))])
            if conv != []:
                convs.append(conv)
            conv = []
            max_utt_len =0
            curr_id = row['B2']
            er_strategies_feats = np.zeros(len(er_strategies))
            er_DAs_feats        = np.zeros(len(er_DAs))
            ee_DAs_feats        = np.zeros(len(ee_DAs))

        utt_dict = {}
        text_df = df3[(df3['B2']==row['B2'])& (df3['Turn']==row['Turn'])& (df3['B4']==row['B4'])]
        row_sent = str(row['Unit']).replace(' ','').strip()
        
        for index, row2 in text_df.iterrows():
            df3_sent = str(row2['Unit']).replace(' ','').strip()
            if row_sent == df3_sent:
        
                utt_dict['utterance'] = row2['Unit']
                utt = normalizeString(row2['Unit'])
                toks = tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[CLS]')) +  tokenizer.convert_tokens_to_ids(tokenizer.tokenize(utt)) + tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[SEP]'))
                max_utt_len = max(max_utt_len, len(toks))
                utt_dict['bert-feat'] = toks
                utt_dict['fine_labels'] = row['fine_labels']
                utt_dict['coarse_labels']= row['coarse_labels']
                utt_dict['resistance_labels']= row['resistance_labels']
                utt_dict['speaker']= row['B4']

                word_list= re.findall(only_words, row2['Unit'].lower().strip())
                vad_feats, affect_feats, emo_feats = create_embeddings(word_list, vads, affects, emos)
                
                for elem in vad_feats:
                    if elem!=elem:
                        import pdb; pdb.set_trace()
                
                liwc_feats= create_liwc(word_list)

                utt_dict['vad_features'] = vad_feats
                utt_dict['affect_features'] = affect_feats
                utt_dict['emo_features'] = emo_feats
                utt_dict['liwc_features'] = [liwc_feats[i] for i in liwc_categories]

                utt_dict['sentiments'] = [row2['pos'], row2['neg'],row2['neu']]
                
                utt_dict['face_acts']  = []
        
                for fact in face_acts:
                    utt_dict['face_acts'].append(row2[fact])

                if row['B4'] == 0:
                    if row['er_label_1'] in er_strategies:
                        er_index = er_strategies.index(row['er_label_1'])
                        er_strategies_feats[er_index]+=1

                    else:
                        er_index = er_DAs.index(row['er_label_1'])
                        er_DAs_feats[er_index]+=1
                else:           
                    ee_index = ee_DAs.index(row['ee_label_1'])
                    ee_DAs_feats[ee_index]+=1

                utt_dict['er_strategies']      = list(er_strategies_feats)
                utt_dict['norm_er_strategies'] = list(normalize_arr(er_strategies_feats))
                utt_dict['er_DAs']             = list(er_DAs_feats)
                utt_dict['norm_er_DAs']        = list(normalize_arr(er_DAs_feats))
                utt_dict['ee_DAs']             = list(ee_DAs_feats)
                utt_dict['norm_ee_DAs']        = list(normalize_arr(ee_DAs_feats))
                if row['B2'] in sincere_donors_ids:
                    utt_dict['donor'] = 1
                else:
                    utt_dict['donor'] = 0
                conv.append(utt_dict)
                break

    for elem in conv:
        elem['bert-feat'].extend([0 for i in range(max_utt_len-len(elem['bert-feat']))])
        
    if conv != []:
        convs.append(conv)

    print(len(convs))
    
    return convs

In [18]:
import json

train_ids_dict =  info_dict['train_ids']
test_ids_dict  =  info_dict['test_ids']


for count in range(0,5):
    print('count {}'.format(count), end ='\r')
    train_ids = train_ids_dict[count]
    test_ids  = test_ids_dict[count]
    
    train= df.loc[df['B2'].isin(train_ids)]
    conv_json = create_conv(train, True)
    with open(DATA_PATH+'/higru_bert_data/train'+str(count)+'.json','w') as handle:
        json.dump(conv_json,handle, indent=4)
    
    test= df.loc[df['B2'].isin(test_ids)]
    conv_json = create_conv(test, False)
    with open(DATA_PATH+'/higru_bert_data/test'+str(count)+'.json','w') as handle:
        json.dump(conv_json,handle, indent=4)


Index(['B2', 'B4', 'Turn', 'Unit', 'er_label_1', 'ee_label_1', 'fine_labels',
       'coarse_labels', 'resistance_labels'],
      dtype='object')


/usr0/home/rdutt/anaconda3/envs/med_ent/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr0/home/rdutt/anaconda3/envs/med_ent/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


226e for 10391
Index(['B2', 'B4', 'Turn', 'Unit', 'er_label_1', 'ee_label_1', 'fine_labels',
       'coarse_labels', 'resistance_labels'],
      dtype='object')
58ne for 10208
Index(['B2', 'B4', 'Turn', 'Unit', 'er_label_1', 'ee_label_1', 'fine_labels',
       'coarse_labels', 'resistance_labels'],
      dtype='object')
227e for 10391
Index(['B2', 'B4', 'Turn', 'Unit', 'er_label_1', 'ee_label_1', 'fine_labels',
       'coarse_labels', 'resistance_labels'],
      dtype='object')
57ne for 10185
Index(['B2', 'B4', 'Turn', 'Unit', 'er_label_1', 'ee_label_1', 'fine_labels',
       'coarse_labels', 'resistance_labels'],
      dtype='object')
226e for 10350
Index(['B2', 'B4', 'Turn', 'Unit', 'er_label_1', 'ee_label_1', 'fine_labels',
       'coarse_labels', 'resistance_labels'],
      dtype='object')
58ne for 10391
Index(['B2', 'B4', 'Turn', 'Unit', 'er_label_1', 'ee_label_1', 'fine_labels',
       'coarse_labels', 'resistance_labels'],
      dtype='object')
225e for 10391
Index(['B2', 'B4', 

In [14]:
for count in range(0,5):
    print('count {}'.format(count), end ='\r')
    train_ids = train_ids_dict[count]
    test_ids  = test_ids_dict[count]
    
    train= df.loc[df['B2'].isin(train_ids)]
    train_labels = set(train['fine_labels'])
    
    test= df.loc[df['B2'].isin(test_ids)]
    test_labels = set(test['fine_labels'])
    
    print(len(train_labels), len(test_labels), len(test_labels - train_labels))
    

10 10 0
10 10 0
10 10 0
10 10 0
10 10 0


In [15]:
feature_dim_dict = {'vad_features': 3, 'affect_features': 4, 'emo_features': 10, 'liwc_features': 64, 'sentiments': 3, 'face_acts': 8, 'norm_er_strategies': 10, 'norm_er_DAs': 17, 'ee_DAs': 23, 'all': 3+4+10+64+3+8+10+17+23}

feature_dim_arr  = [i for i in feature_dim_dict if i!='all']
print(feature_dim_arr)

['vad_features', 'affect_features', 'emo_features', 'liwc_features', 'sentiments', 'face_acts', 'norm_er_strategies', 'norm_er_DAs', 'ee_DAs']
